### TF = вхождение термина / все слова в документе
### IDF = -log(число документов, в которых есть термин / все документы)

### Из массива текстов мы делаем векторы

In [ ]:
import sklearn # библиотека для машинного обучения

In [ ]:
from sklearn.datasets import fetch_20newsgroups

In [ ]:
# Уже готовый датасет

dataset = fetch_20newsgroups(subset ='all', 
                             categories=['alt.atheism',
                                        'comp.graphics',
                                        'comp.os.ms-windows.misc'],
                             shuffle=True, random_state=42)

In [ ]:
len(dataset['data']) # список строчек с сообщениями

In [ ]:
dataset['target_names']

In [ ]:
import numpy as np

In [ ]:
np.bincount(dataset['target']) # Количество текстов в каждой категории

### Далее мы векторизуем каждый текст, создаём таблицу

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
vectorizer = TfidfVectorizer(max_df=0.5, min_df=2, # Если слово встречается болше чем в половине документов или менее, чем в двух, то мы его игнорируем 
                            stop_words='english',
                            use_idf=True)

In [ ]:
X = vectorizer.fit_transform(dataset['data'])

In [ ]:
X

In [ ]:
feature_names = vectorizer.get_feature_names() # Названия столбцов в таблице, feature – это название столбца

In [ ]:
feature_names.index('data') # Инднекс feature, ищем слово 'data'

In [ ]:
(X[:, 7826] > 0).sum() # указываем номер столбца, в 180 текстах встречается слово 'data'

In [ ]:
X # Это огромная матрица

In [ ]:
from sklearn.cluster import KMeans

km = KMeans(n_clusters=3, init='k-means++',
           n_init=1, max_iter=300)

In [ ]:
km.fit(X)

In [ ]:
len(km.labels_)

In [ ]:
km.labels_[:10] # Кластеризованные тексты, номера кластеров, они могут не совпадать с номерами категории

In [ ]:
dataset['target'][:10] # Настоящие категории текстов

In [ ]:
from sklearn.metrics import homogeneity_score

In [ ]:
homogeneity_score(dataset['target'], km.labels_) # Проверяем соответствие

In [ ]:
from sklearn.decomposition import TruncatedSVD

In [ ]:
pca = TruncatedSVD()

In [ ]:
transformed_texts = pca.fit_transform(X)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline

In [ ]:
plt.scatter(transformed_texts[:,0], 
            transformed_texts[:,1],
            c=km.labels_, alpha=0.8, cmap='rainbow')
plt.colorbar()

In [ ]:
dataset['target_names']

### Общеупотребительная лексика отсеивается при помощи TF IDF

In [ ]:
centers = pca.transform(km.cluster_centers_)

In [ ]:
# Визуализируем центроиды

plt.scatter(transformed_texts[: ,0],
            transformed_texts[: ,1], 
            c = km.labels_,
           alpha=0.2) #а это кластеризация
centers = pca.transform(km.cluster_centers_) 
plt.scatter(centers[:,0],centers[:,1],c = [0,1,2],cmap = "rainbow", edgecolor = "black")